In [1]:
import pandas as pd
import geopandas
import folium
import numpy as np
import datetime as dt
from datetime import datetime, timedelta
from shapely.geometry import Point, Polygon, CAP_STYLE, box

In [55]:
zones={
'Mel North':Polygon([(144.8628073558594, -37.7212887643389), (145.0111227855469, -37.71585722493781), (145.00974949453126, -37.78752147312994), (144.8628073558594, -37.79294775423262)]),
'Mel West': Polygon([(144.73509129140626, -37.75821267002394), (144.8573141917969, -37.75821267002394), (144.86006077382814, -37.848273056101036), (144.73509129140626, -37.86020052689999)]),
'Mel East':Polygon([(145.05781468007814, -37.78969203338907), (145.20201023671876, -37.79294775423262), (145.193770490625, -37.888385058604605), (145.05918797109376, -37.89055265284611)]),
'Mel Center':Polygon([(144.87791355703126, -37.804884169931675), (145.03034885976564, -37.80162897512093), (145.03584202382814, -37.86020052689999), (144.875166975, -37.85152619406298)]),
'Mel SouthEast':Polygon([(145.00288303945314, -37.900306037210385), (145.14570530507814, -37.90789110935228), (145.18003758046876, -37.99560434902989), (145.0330954417969, -37.99127528948802)]),                     
'Syd North':Polygon([(151.1512687398634, -33.7935992898409), (151.2281730367384, -33.79645241440158), (151.24122619628906, -33.84333046657819), (151.16294171349622, -33.84180431578851)]),
'Syd Center':Polygon([(151.16912152306654, -33.856631164603094), (151.20414044396497, -33.87402091845947), (151.20414044396497, -33.91078430372838), (151.1553886129103, -33.90451569998407)]),
'Syd East':Polygon([(151.25769879357435, -33.874591014351736), (151.20414044396497, -33.95664505114222), (151.2439658834181, -34.00390417629284), (151.28585125939466, -33.85121396050401)]),
'Syd West':Polygon([(151.0523917867384, -33.87060026313806), (151.15401532189466, -33.87060026313806), (151.13272931115247, -33.924744536934675), (151.02767254845716, -33.92417477627145)]),
'Syd Parramatta':Polygon([(150.93291546837904, -33.767346082697074), (151.0523917867384, -33.776478545694935), (151.0523917867384, -33.866609325339375), (150.90476300255872, -33.87060026313806)]),
'Syd South':Polygon([(151.08191754357435, -33.93670863052086), (151.2000205709181, -33.92702354147929), (151.1553886129103, -34.0642209857176), (151.03797223107435, -34.03691302642577)]),
'Bri North':Polygon([(152.96972916835938, -27.32627690929374), (153.08165238613282, -27.331156966171314), (153.12628434414063, -27.421399244879808), (152.9759089779297, -27.426884576089662)]),      
'Bri Center':Polygon([(152.97865555996094, -27.447604478804557), (153.15031693691407, -27.426884576089662), (153.19769547695313, -27.49938720750099), (152.9765956234375, -27.542013525008592)]),
'Bri South':Polygon([(152.9820887875, -27.5505368052517), (153.2011287044922, -27.542013525008592), (153.2121150326172, -27.619916025981073), (152.98964188808594, -27.661279210523194)]),
'Bri West':Polygon([(152.79806779140625, -27.538360488181283), (152.96423600429688, -27.503650582690096), (152.97453568691407, -27.65276453925091), (152.80424760097657, -27.690467333539093)])
}
pivot_z = {'zone':zones.keys(), 'geometry':zones.values()}
zones_gdf = geopandas.GeoDataFrame(pivot_z, crs="EPSG:4326" )

#make it square zone
square = [Polygon([(l,b),(r,b),(r,t),(l,t)]) for l, b, r, t in zones_gdf.geometry.bounds.values.tolist()] 
square_gdf = geopandas.GeoDataFrame(zones_gdf['zone'], geometry=square, crs="EPSG:4326")

tost=square_gdf.to_crs({'init': 'epsg:32633'})
square_gdf['radius']=np.sqrt( tost['geometry'].area/ 10**6/np.pi)
square_gdf['center_longitude']=square_gdf.geometry.centroid.x
square_gdf['center_latitude']=square_gdf.geometry.centroid.y

c:\program files\anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
c:\program files\anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

c:\program files\anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [33]:
square_gdf.to_file('target_zones.geojson', driver="GeoJSON")

In [53]:
square_gdf.to_csv('target_zones.csv')

In [34]:
square_gdf=geopandas.read_file('target_zones.geojson')

## Census data: Income, Age (SA2)

In [35]:
df=pd.read_csv('SA2-P02_Selected_Medians_and_Averages-Census_2016.csv')
df.columns = [x.strip().upper() for x in df.columns]
df['SA2_MAIN16'] = df['SA2_MAIN16'].astype(str)

In [ ]:
#http://apps.headwallphotonics.com/

In [36]:
geo_data =geopandas.read_file('./shapefile/sa2_low_resolution.geojson')
geo_data = geo_data[~geo_data.geometry.isna()]
loc = Point(144,-37.8) #long, lat
loc_buffer=loc.buffer(0.5, cap_style=CAP_STYLE.square)
geo_data['in'] = geo_data.geometry.contains(loc)
geo_data['contains'] = geo_data.geometry.apply(lambda x: loc_buffer.contains(x))

geo_data[geo_data['in']]

,SA2_MAIN16,SA2_NAME16,geometry,in,contains
589,201021012,Gordon (Vic.),"POLYGON ((143.94409 -37.57381, 143.94168 -37.5...",True,True


In [37]:
sa2_to_zones =geopandas.tools.sjoin(square_gdf, geo_data, how='left')
zone_income_age = pd.merge(sa2_to_zones, df).groupby(['zone']).mean()[['MEDIAN_AGE_PERSONS','MEDIAN_TOT_FAM_INC_WEEKLY',
       'AVERAGE_HOUSEHOLD_SIZE', 'MEDIAN_TOT_PRSNL_INC_WEEKLY']]

c:\program files\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:4283

  """Entry point for launching an IPython kernel.


In [38]:
m = folium.Map(location=[-23, 133], zoom_start=5)

folium.Choropleth(
    geo_data=square_gdf,
    name='choropleth',
    data=zone_income_age.reset_index(),
    columns=['zone', 'MEDIAN_TOT_FAM_INC_WEEKLY'],
    key_on='feature.properties.zone',
    nan_fill_color='white',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Median Total Family Income'
).add_to(m)
folium.GeoJson(data=square_gdf).add_to(m)
folium.LayerControl().add_to(m)
m

## Religion

In [39]:
religon=pd.read_csv('SA3-G14_Religious_Affiliation_by_Sex-Census_2016.csv')
religon.columns = [x.strip().upper() for x in religon.columns]
religon = religon[[ 'SA3_NAME16','TOT_P','CHRISTIANITY_TOT_P','ISLAM_P','HINDUISM_P', 'SB_OSB_NRA_NR_P']]

In [40]:
geo_sa3 =geopandas.read_file('./shapefile/sa3_low_resolution.geojson')
geo_sa3 = geo_sa3[~geo_sa3.geometry.isna()]

In [41]:
sa3_to_zones =geopandas.tools.sjoin(square_gdf, geo_sa3, how='left')
zone_religon = pd.merge(sa3_to_zones, religon, on='SA3_NAME16').groupby(['zone']).sum()

c:\program files\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:4283

  """Entry point for launching an IPython kernel.


In [42]:
zone_religon.columns

Index(['radius', 'center_longitude', 'center_latitude', 'index_right', 'TOT_P',
       'CHRISTIANITY_TOT_P', 'ISLAM_P', 'HINDUISM_P', 'SB_OSB_NRA_NR_P'],
      dtype='object')

In [43]:
zone_religon['ratio_christian'] = zone_religon.CHRISTIANITY_TOT_P/zone_religon.TOT_P
zone_religon['ratio_islam'] = zone_religon.ISLAM_P/zone_religon.TOT_P
zone_religon['ratio_hindu'] = zone_religon.HINDUISM_P/zone_religon.TOT_P
zone_religon['ratio_no_religon'] = zone_religon.SB_OSB_NRA_NR_P/zone_religon.TOT_P


In [44]:
m = folium.Map(location=[-23, 133], zoom_start=5)

folium.Choropleth(
    geo_data=square_gdf,
    name='choropleth',
    data=zone_religon.reset_index(),
    columns=['zone', 'ratio_christian'],
    key_on='feature.properties.zone',
    nan_fill_color='white',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Ratio of Christian in total population'
).add_to(m)
folium.GeoJson(data=square_gdf).add_to(m)
folium.LayerControl().add_to(m)
m

## Education

In [45]:
edu=pd.read_csv('SA3-based_B16A_Highest_Year_of_School_completed_by_Age_by_Sex_as_at_2011-08-11.csv')
edu.columns = [x.strip().upper() for x in edu.columns]
edu = edu[['M_DNGTS_TOT',
 'M_Y8B_TOT',
 'F_Y8B_TOT',
 'F_DNGTS_TOT',
 'F_Y9E_TOT',
 'P_Y9E_TOT',
 'P_Y10E_TOT',
 'P_Y11E_TOT',
 'P_Y12E_TOT','SA3_NAME11']]
zone_edu = pd.merge(sa3_to_zones, edu, left_on='SA3_NAME16', right_on='SA3_NAME11').groupby(['zone']).sum()

In [46]:
zone_edu['P_TOT']=zone_edu[['M_DNGTS_TOT',
 'M_Y8B_TOT',
 'F_Y8B_TOT',
 'F_DNGTS_TOT',
 'F_Y9E_TOT',
 'P_Y9E_TOT',
 'P_Y10E_TOT',
 'P_Y11E_TOT',
 'P_Y12E_TOT']].sum(axis=1)
zone_edu['avg_study_yrs'] = zone_edu[[
 'M_Y8B_TOT',
 'F_Y8B_TOT',
 'P_Y9E_TOT',
 'P_Y10E_TOT',
 'P_Y11E_TOT',
 'P_Y12E_TOT','P_TOT']].apply(lambda x: (x[0]*4.0 +x[1]*4+x[2]*9+x[3]*10+x[4]*11+x[5]*12)/x[6], axis=1)

In [47]:
zone_edu

,radius,center_longitude,center_latitude,index_right,M_DNGTS_TOT,M_Y8B_TOT,F_Y8B_TOT,F_DNGTS_TOT,F_Y9E_TOT,P_Y9E_TOT,P_Y10E_TOT,P_Y11E_TOT,P_Y12E_TOT,P_TOT,avg_study_yrs
zone,,,,,,,,,,,,,,,
Adelaide,979.712602,2910.589563,-733.345830,5381,3662,28105,34631,5101,25411,49074,161442,177701,493179,978306,10.405629
Ballarat,213.078744,719.250030,-187.749995,483,329,5708,5902,259,6279,13610,29758,25076,64209,151130,10.010283
Brisbane,1541.474343,5355.737520,-961.454375,6591,3840,36009,42822,4475,28998,59234,317507,97598,834019,1424502,10.603921
Bunbury,180.052031,346.899993,-99.999996,831,227,2975,2667,191,3189,7012,36300,14522,45180,112263,10.248951
Canberra,293.815531,1043.902986,-246.974007,2060,529,3855,4742,735,5198,9533,45456,13180,193960,277188,10.993423
Geelong,210.289818,721.750030,-190.750010,546,987,9882,11657,1290,11285,22696,54799,49580,156934,319110,10.237843
Hobart,345.244855,1325.924973,-385.924986,2853,382,4595,5135,370,6890,13839,55257,15761,82909,185138,10.177921
Melbourne,1597.451201,5507.965366,-1437.955530,4895,15822,79254,106015,23372,64747,129607,334948,300145,1667948,2721858,10.497972
Newcastle,295.122253,1062.250000,-230.416676,168,1275,15345,16851,1127,29993,56087,183498,32620,194313,531109,9.713869


## Overseas vs Local

In [48]:
overseas=pd.read_csv('./ABS_-_Data_by_Region_-_Persons_Born_Overseas__SA3__2011-2016.csv')
overseas.columns = [x.strip().upper() for x in overseas.columns]
overseas=overseas[['POPULATION_OF_PERSONS_BORN_OVERSEAS_CENSUS_PERSONS_NUM','SA3_CODE_2016','SA3_NAME_2016']]
zone_overseas = pd.merge(sa3_to_zones, overseas, left_on='SA3_NAME16', right_on='SA3_NAME_2016').groupby(['zone']).sum()

In [49]:
zone_overseas = pd.merge(zone_overseas, zone_religon['TOT_P'], left_index=True, right_index=True)
zone_overseas['rate_overseas'] = zone_overseas.POPULATION_OF_PERSONS_BORN_OVERSEAS_CENSUS_PERSONS_NUM/zone_overseas.TOT_P

## Final

In [50]:
zone_stats = pd.merge(
pd.merge(
    pd.merge(zone_overseas['rate_overseas'], zone_edu['avg_study_yrs'], left_index=True, right_index=True),
    zone_religon[['ratio_christian','ratio_islam','ratio_hindu','ratio_no_religon']], left_index=True, right_index=True),
    zone_income_age[['MEDIAN_AGE_PERSONS','MEDIAN_TOT_FAM_INC_WEEKLY']], left_index=True, right_index=True)

In [51]:
zone_stats.reset_index().to_csv('zone_stats.csv')

In [52]:
zone_stats

,rate_overseas,avg_study_yrs,ratio_christian,ratio_islam,ratio_hindu,ratio_no_religon,MEDIAN_AGE_PERSONS,MEDIAN_TOT_FAM_INC_WEEKLY
zone,,,,,,,,
Adelaide,0.255331,10.405629,0.483511,0.019678,0.016125,0.356767,37.991150,1572.530973
Ballarat,0.101485,10.010283,0.495737,0.002560,0.003646,0.376043,42.235294,1480.823529
Brisbane,0.266866,10.603921,0.534995,0.015115,0.014942,0.307790,35.111111,1914.604444
Bunbury,0.187139,10.248951,0.509783,0.004248,0.002949,0.357923,40.750000,1707.937500
Canberra,0.229801,10.993423,0.495510,0.020805,0.021011,0.332959,34.601449,2069.637681
Geelong,0.262233,10.237843,0.494498,0.035676,0.039501,0.298252,40.115385,1632.961538
Hobart,0.135035,10.177921,0.475230,0.006163,0.006467,0.396981,40.348837,1498.837209
Melbourne,0.349824,10.497972,0.461262,0.044549,0.030569,0.303422,36.183267,1914.095618
Newcastle,0.107507,9.713869,0.614728,0.004470,0.003756,0.278221,40.784314,1622.745098


## Zone by Cities

In [24]:
zones={
'Geelong':(144.350006,-38.150002),
'Melbourne':(144.946457,-37.840935),
'Perth':(115.857048,-31.953512),
'Brisbane':(153.021072,-27.470125),
'Sydney':(151.2099,-33.865143),
'Hobart':(147.324997,-42.880554),
'Canberra':(149.128998,-35.282001),
'Newcastle':(151.75,-32.916668),
'Adelaide':(138.599503,-34.92123),
'Bunbury':(115.633331,-33.333332),
'Ballarat':(143.850006,-37.549999)
}
pivot_z = {'zone':zones.keys(), 'center':zones.values()}

In [25]:
pivot_df = pd.DataFrame(pivot_z)

In [26]:
square = [Point(t).buffer(0.3, cap_style=CAP_STYLE.square) for t in pivot_df.center] 
square_gdf = geopandas.GeoDataFrame(pivot_df, geometry=square, crs="EPSG:4326")
tost=square_gdf.to_crs({'init': 'epsg:32633'})
square_gdf['radius']=np.sqrt( tost['geometry'].area/ 10**6/np.pi)
square_gdf['center_longitude']=square_gdf.geometry.centroid.x
square_gdf['center_latitude']=square_gdf.geometry.centroid.y

c:\program files\anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
c:\program files\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """
c:\program files\anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


In [32]:
square_gdf.drop('center', axis=1, inplace=True)